# 0. Install

เราจะต้อง Install kaggle เพื่อ Download Dataset, geopandas เพื่อใช้ในการวิเคราะห์ข้อมูล geospatial, folium เพื่อแสดงแผ่นที่ (ถ้ายังไม่ได้ Install ให้ uncomment)

In [0]:
# ! pip install geopandas
# ! pip install git+https://github.com/python-visualization/folium
# ! pip install kaggle --upgrade

# 1. Import Library

Import folium Library เพื่อใช้ในการพล็อตแผนที่แบบ Interactive

In [0]:
import pandas as pd
import geopandas as gpd

import folium
from folium import *
from folium.plugins import *

import os
from pathlib import Path

# 2. เตรียม Path สำหรับดาวน์โหลดข้อมูล

กำหนด path ของ Config File และ Dataset ว่าจะอยู่ใน Google Drive ถ้าเราใช้ Google Colab หรือ อยู่ใน HOME ถ้าเราใช้ VM ธรรมดา และกำหนด Environment Variable ไปยังโฟลเดอร์ที่เก็บ kaggle.json

ในกรณีใช้ Colab ให้ Mount Google Drive เพื่อดึง Config File มาจาก Google Drive ส่วนตัวของเรา เมื่อเรารัน Cell ด้านล่างจะมีลิงค์ปรากฎขึ้นมาให้เรา Login กด Approve แล้ว Copy Authorization Code มาใส่ในช่องด้านล่าง แล้วกด Enter

In [0]:
dataset = 'alexisbcook/geospatial-learn-course-data'

## Google Colab
# config_path = '/content/drive'
# data_path = '/content/datasets/' + dataset
# from google.colab import drive
# drive.mount(config_path)
# os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/My Drive/.kaggle"

## GCP VM
config_path = Path(os.getenv("HOME"))
data_path = config_path/"datasets"/dataset
data_path.mkdir(parents=True, exist_ok=True)
os.environ['KAGGLE_CONFIG_DIR'] = f"{config_path}/.kaggle"

# 3. Dataset

สั่งดาวน์โหลด Dataset พร้อมทั้ง unzip ไว้ใน data_path

In [0]:
!kaggle datasets download {dataset} -p "{data_path}" --unzip

 93%|█████████████████████████████████████▉   | 216M/233M [00:02<00:00, 108MB/s]
100%|█████████████████████████████████████████| 233M/233M [00:02<00:00, 104MB/s]


ลองโหลด Shape File ของเปลือกโลกภาคพื้นทวีป ขึ้นมาแสดง 5 แถวแรก โดยแปลง Feature coordinates จาก Feature geometry coords

In [0]:
plate_boundaries = gpd.read_file(data_path/"Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(-5.444200361999947, 133.6808931800001), (-5...."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(-7.760600482999962, 125.47879802900002), (-7..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(19.817899819000047, 120.09999798800004), (19..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(26.175899215000072, 143.20620700100005), (26..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(0.41880004000006466, 132.8273013480001), (0...."


โหลดข้อมูลแผ่นดินไหวทั่วโลก ตั้งแต่ปี 1970-2014 แล้วแสดง 5 แถวแรก

In [0]:
earthquakes = pd.read_csv(data_path/"earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


เนื่องจาก folium บน Colab เมื่อ Data เยอะเกินจะ Error เราจะจำกัดอยู่ที่ไม่เกิน 3000 Records 

In [0]:
## Colab
# earthquakes = earthquakes.sample(3000)

In [0]:
from IPython.display import IFrame, HTML

def embed_map(m, file_name):    
    ## VM
    m.save(file_name)
    return IFrame(src=file_name, width='100%', height='500px')
    # # Colab
    # return m

In [0]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Your code here: Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=16).add_to(m_1)

# Show the map
embed_map(m_1, 'm_1.html')

In [0]:
<iframe width="100%" height="500" src="https://www.bualabs.com/wp-content/uploads/2019/10/m_1.html" frameborder="0" allowfullscreen></iframe>

In [0]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
# for i in range(len(plate_boundaries)):
#     folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
earthquakes['marker_color'] = pd.cut(earthquakes['Depth'], bins=4, 
                              labels=['yellow', 'orange', 'red', 'purple'])

# Circle(location=[36, 135], radius=50000, color='red').add_to(m_2)
# Circle(location=[38, 135], radius=50000, color='green').add_to(m_2)
# Circle(location=[33, 135], radius=50000, color='blue').add_to(m_2)

# Your code here: Add a map to visualize earthquake depth
for idx, e in earthquakes.iterrows():
    Circle(location=[e['Latitude'], e['Longitude']], radius=15000, color=e['marker_color']).add_to(m_2)
    # print((e['Latitude'], e['Longitude']))
    # Circle(location=(e['Latitude'], e['Longitude']), radius=5000, color='red').add_to(m_2)
    

embed_map(m_2, 'm_2.html')

<iframe width="100%" height="500" src="https://www.bualabs.com/wp-content/uploads/2019/10/m_2.html" frameborder="0" allowfullscreen></iframe>

Can you detect a relationship between proximity to a plate boundary and earthquake depth?  Does this pattern hold globally?  In Japan?

In the northern half of Japan, it does appear that earthquakes closer to plate boundaries tend to be shallower (and earthquakes farther from plate boundaries are deeper). This pattern is repeated in other locations, such as the western coast of South America. But, it does not hold everywhere (for instance, in China, Mongolia, and Russia).

Which prefectures have high population density?



In [0]:
prefectures = gpd.read_file(data_path/"japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


In [0]:
# DataFrame containing population of each prefecture
population = pd.read_csv(data_path/"japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


In [0]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a choropleth map to visualize population density
Choropleth(geo_data=prefectures.__geo_interface__, data=stats.density, key_on='feature.id', 
           fill_color='YlGnBu', 
           legend_name='population density').add_to(m_3)

embed_map(m_3, 'm_3.html')

<iframe width="100%" height="500" src="https://www.bualabs.com/wp-content/uploads/2019/10/m_3.html" frameborder="0" allowfullscreen></iframe>

Which three prefectures have relatively higher density than the others?  Are they spread throughout the country, or all located in roughly the same geographical region?  (*If you're unfamiliar with Japanese geography, you might find [this map](https://en.wikipedia.org/wiki/Prefectures_of_Japan) useful to answer the questions.)

Tokyo, Kanagawa, and Osaka have the highest population density. All of these prefectures are located in central Japan, and Tokyo and Kanagawa are adjacent.

4) Which high-density prefecture is prone to high-magnitude earthquakes?

In [0]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a map
Choropleth(geo_data=prefectures.geometry.__geo_interface__, data=stats.density, key_on='feature.id', 
           fill_color='BuPu', 
           legend_name='population density and earthquake magnitude').add_to(m_4)

earthquakes['marker_color'] = pd.cut(earthquakes['Magnitude'], bins=4, 
                              labels=['yellow', 'orange', 'red', 'purple'])

# Your code here: Add a map to visualize earthquake depth
for idx, e in earthquakes.iterrows():    
    Circle(location=[e['Latitude'], e['Longitude']], 
           popup=("{} ({})").format(e['Magnitude'],e['DateTime'].year),
           radius=e['Magnitude']**5.5,           
           color=e['marker_color']).add_to(m_4)

embed_map(m_4, 'm_4.html')

<iframe width="100%" height="500" src="https://www.bualabs.com/wp-content/uploads/2019/10/m_4.html" frameborder="0" allowfullscreen></iframe>

Which prefecture do you recommend for extra earthquake reinforcement?

While there's no clear, single answer to this question, there are a few reasonable options. Tokyo is by far the most densely populated prefecture and has also experienced a number of earthquakes. Osaka is relatively less densely populated, but experienced an earthquake that was relatively stronger than those near Tokyo. And, the long coast of Kanagawa (in addition to its high density and the historical proximity of strong earthquakes) might lead us to worry about the added potential tsunami risk.

# Credit

* https://www.kaggle.com/alexisbcook/interactive-maps
* https://python-visualization.github.io/folium/modules.html
* https://www.bualabs.com/archives/751/multi-label-image-classification-satellite-imagery-deep-learning-machine-learning-image-classification-ep-5/
* https://en.wikipedia.org/wiki/Prefectures_of_Japan